## Import libraries

In [1]:
import cassandra
from cassandra.cluster import Cluster

## Create a connection to the database

In [2]:
try:
    cluster = Cluster(["127.0.0.1"])
    session = cluster.connect()
except Exception as e:
    print(e)

## Create a keyspace to do the work in and connect to it

In [3]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity_keyspace
    WITH REPLICATION = 
    {'class': 'SimpleStrategy', 'replication_factor': 1}
    """)
    session.set_keyspace("udacity_keyspace")
except Exception as e:
    print(e)

## Music library

Let's imagine we would like to start creating a new Music Library of albums.  
We want to ask 4 question of our data:
1. Give me every album in my music library that was released in a 1965 year.
2. Give me the album that is in my music library that was released in 1965 by "The Beatles".
3. Give me all the albums released in a given year that were made in London.
4. Give me the city where the album "Rubber Soul" was recorded.

Here is our collection of data:

![music_library_cassandra_db](images/music_library_cassandra_db_2.png)


**How should we model this data? What should be our Primary Key and Partition Key? Since our data is looking for the YEAR let's start with that. From there we will add clustering columns on Artist Name and Album Name.**

## Data modeling

In [4]:
# Create table
create_music_library = """
CREATE TABLE IF NOT EXISTS music_library (
    year INT,
    artist_name TEXT,
    album_name TEXT,
    city TEXT,
    PRIMARY KEY (year, artist_name, album_name)
);
"""

try:
    session.execute(create_music_library)
except Exception as e:
    print(e)
    
# Insert data
insert_music_library = """
INSERT INTO music_library (year, artist_name, album_name, city)
VALUES (%s, %s, %s, %s);
"""

music_library_data = [
    (1965, "The Beatles", "Rubber Soul", "Oxford"),
    (1970, "The Beatles", "Let it Be", "Liverpool"),
    (1966, "The Monkees", "The Monkees", "Los Angeles"),
    (1970, "The Carpenters", "Close To You", "San Diego"),
    (1965, "The Who", "My Generation", "London")
]

try:
    for item in music_library_data:
        session.execute(insert_music_library, item)
except Exception as e:
    print(e)

## Validate our model with the 4 queries

### Query 1
**Give me every album in my music library that was released in a 1965 year.**

In [5]:
query = """
SELECT *
FROM music_library
WHERE year = 1965;
"""

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.year, row.artist_name, row.album_name, row.city)

1965 The Beatles Rubber Soul Oxford
1965 The Who My Generation London


### Query 2
**Give me the album that is in my music library that was released in 1965 by "The Beatles".**

In [6]:
query = """
SELECT *
FROM music_library
WHERE year = 1965
AND artist_name = 'The Beatles';
"""

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.year, row.artist_name, row.album_name, row.city)

1965 The Beatles Rubber Soul Oxford


### Query 3
**Give me all the albums released in a given year that were made in London.**

In [7]:
query = """
SELECT *
FROM music_library
WHERE year = 1965
AND city = 'London';
"""

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.year, row.artist_name, row.album_name, row.city)

Error from server: code=2200 [Invalid query] message="Cannot execute this query as it might involve data filtering and thus may have unpredictable performance. If you want to execute this query despite the performance unpredictability, use ALLOW FILTERING"


**ERROR! We can't try to access a column or a clustering column if we haven't defind the other clustering columns**  
So in this case, if we want to filter on `city` (normal column), we have to use first `year` (partition key) and `artist_name` and `album_name` in that order (clustering columns).  
Let's try it a different way.

In [8]:
query = """
SELECT *
FROM music_library
WHERE year = 1965
AND artist_name = 'The Who'
AND album_name = 'My Generation';
"""

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.year, row.artist_name, row.album_name, row.city)

1965 The Who My Generation London


### Query 4
**Give me the city were the album "Rubber Soul" was recorded.**

In [9]:
query = """
SELECT city
FROM music_library
WHERE year = 1965
AND artist_name = 'The Beatles'
AND album_name = 'Rubber Soul';
"""

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.city)

Oxford


## Drop the table

In [10]:
try:
    session.execute("DROP TABLE music_library;")
except Exception as e:
    print(e)

## Close session and cluster connection

In [11]:
session.shutdown()
cluster.shutdown()